In [ ]:
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
# Install required packages and setup display
!pip install ultralytics
%matplotlib inline
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [20, 10]
%config InlineBackend.figure_format = 'retina'

In [ ]:
#Object detection using Semantic Segmentation

import cv2
import numpy as np
import matplotlib.pyplot as plt
import torch
import os
from pathlib import Path
import logging
from datetime import datetime

# Configure logging
logging.basicConfig(level=logging.INFO,
                   format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# Print current directory contents to verify images
print("Current directory contents:")
print(os.listdir())

class NightVisionSystem:
    def __init__(self):
        """Initialize the night vision system"""
        logger.info("Loading YOLOv5 model...")
        try:
            self.model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)
            self.model.conf = 0.5
            self.model.iou = 0.45
            self.model.classes = None
            self.model.max_det = 1000
        except Exception as e:
            logger.error(f"Error loading YOLOv5 model: {str(e)}")
            raise

        self.target_classes = {
            'person': 'person',
            'car': 'vehicle',
            'truck': 'vehicle',
            'motorcycle': 'vehicle',
            'bus': 'vehicle',
            'bicycle': 'vehicle',
            'dog': 'animal',
            'cat': 'animal',
            'horse': 'animal',
            'cow': 'animal',
            'sheep': 'animal'
        }

        self.object_colors = {
            'person': (0, 255, 0),
            'car': (255, 0, 0),
            'truck': (255, 127, 0),
            'motorcycle': (255, 0, 127),
            'bus': (255, 127, 127),
            'bicycle': (200, 200, 0),
            'dog': (0, 0, 255),
            'cat': (0, 127, 255),
            'horse': (0, 255, 255),
            'cow': (127, 0, 255),
            'sheep': (127, 127, 255)
        }

        self.legend_start_x = 10
        self.legend_start_y = 30
        self.legend_entry_height = 25
        self.legend_text_offset = 30

    def enhance_image(self, image):
        r_channel, g_channel, b_channel = cv2.split(image)
        r_eq = cv2.equalizeHist(r_channel)
        g_eq = cv2.equalizeHist(g_channel)
        b_eq = cv2.equalizeHist(b_channel)
        hist_eq_image = cv2.merge((r_eq, g_eq, b_eq))

        lab = cv2.cvtColor(hist_eq_image, cv2.COLOR_BGR2LAB)
        l_channel, a_channel, b_channel = cv2.split(lab)
        clahe = cv2.createCLAHE(clipLimit=10.0, tileGridSize=(10, 10))
        l_channel_eq = clahe.apply(l_channel)
        lab_eq = cv2.merge((l_channel_eq, a_channel, b_channel))
        clahe_image = cv2.cvtColor(lab_eq, cv2.COLOR_LAB2BGR)

        bilateral_filtered = cv2.bilateralFilter(clahe_image, d=2, sigmaColor=50, sigmaSpace=50)

        gamma = 1.2
        gamma_corrected = np.array(255 * (bilateral_filtered / 255) ** gamma, dtype='uint8')

        gaussian_blur = cv2.GaussianBlur(gamma_corrected, (0, 0), 2.0)
        enhanced_image = cv2.addWeighted(gamma_corrected, 2, gaussian_blur, -1, 0)

        return enhanced_image

    def detect_objects(self, image):
        rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = self.model(rgb_image)
        detections = []

        for pred in results.xyxy[0]:
            x1, y1, x2, y2, conf, cls = pred.cpu().numpy()
            class_name = self.model.names[int(cls)]

            if class_name in self.target_classes:
                category = self.target_classes[class_name]
                detections.append({
                    'category': category,
                    'class': class_name,
                    'confidence': float(conf),
                    'bbox': (int(x1), int(y1), int(x2), int(y2))
                })

        return detections

    def add_color_legend(self, image, used_classes):
        legend_width = 180
        legend_height = len(used_classes) * self.legend_entry_height + 10

        overlay = image.copy()
        cv2.rectangle(overlay, (5, 5),
                     (5 + legend_width, 5 + legend_height),
                     (0, 0, 0), -1)
        cv2.addWeighted(overlay, 0.7, image, 0.3, 0, image)

        cv2.putText(image, "Detected Objects:",
                   (self.legend_start_x, 20),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)

        y_position = self.legend_start_y
        for class_name in sorted(used_classes):
            color = self.object_colors[class_name]

            cv2.rectangle(image,
                         (self.legend_start_x, y_position - 10),
                         (self.legend_start_x + 20, y_position + 10),
                         color, -1)

            cv2.putText(image, class_name,
                       (self.legend_start_x + self.legend_text_offset, y_position + 5),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)

            y_position += self.legend_entry_height

    def visualize_detections(self, image, detections):
        result_image = image.copy()
        used_classes = set()

        for det in detections:
            x1, y1, x2, y2 = det['bbox']
            class_name = det['class']
            color = self.object_colors[class_name]
            used_classes.add(class_name)

            cv2.rectangle(result_image, (x1, y1), (x2, y2), color, 2)

            label = f"{class_name} ({det['confidence']:.2f})"
            label_size, _ = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 2)

            cv2.rectangle(result_image,
                        (x1, y1 - label_size[1] - 10),
                        (x1 + label_size[0], y1),
                        color, -1)

            cv2.putText(result_image, label, (x1, y1-5),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)

        self.add_color_legend(result_image, used_classes)
        return result_image

    def process_image(self, image_path):
        try:
            image = cv2.imread(str(image_path))
            if image is None:
                raise ValueError(f"Could not read image: {image_path}")

            enhanced_image = self.enhance_image(image)
            detections = self.detect_objects(enhanced_image)
            result_image = self.visualize_detections(enhanced_image, detections)

            return {
                'original': image,
                'enhanced': enhanced_image,
                'result': result_image,
                'detections': detections
            }
        except Exception as e:
            logger.error(f"Error processing image {image_path}: {str(e)}")
            raise

# Initialize system and process images
system = NightVisionSystem()
image_paths = ['0.png', '1.png', '2.png', '3.png', '4.png']

# Create a single figure for all images
plt.figure(figsize=(20, 5 * len(image_paths)))

for idx, image_path in enumerate(image_paths, 1):
    try:
        print(f"\nProcessing Image {idx}:")
        results = system.process_image(image_path)

        # Plot in a single row
        base_idx = (idx - 1) * 3 + 1

        # Original image
        plt.subplot(len(image_paths), 3, base_idx)
        plt.imshow(cv2.cvtColor(results['original'], cv2.COLOR_BGR2RGB))
        plt.title(f'Original Image {idx}')
        plt.axis('off')

        # Enhanced image
        plt.subplot(len(image_paths), 3, base_idx + 1)
        plt.imshow(cv2.cvtColor(results['enhanced'], cv2.COLOR_BGR2RGB))
        plt.title(f'Enhanced Image {idx}')
        plt.axis('off')

        # Detection results
        plt.subplot(len(image_paths), 3, base_idx + 2)
        plt.imshow(cv2.cvtColor(results['result'], cv2.COLOR_BGR2RGB))
        plt.title(f'Detected Objects {idx} ({len(results["detections"])} objects)')
        plt.axis('off')

        # Print detection summary
        print(f"Detections in Image {idx}:")
        categories = {}
        for det in results['detections']:
            cat = det['category']
            categories[cat] = categories.get(cat, 0) + 1
        for cat, count in categories.items():
            print(f"- {cat}: {count}")

    except Exception as e:
        logger.error(f"Failed to process {image_path}: {str(e)}")
        print(f"Error processing Image {idx}: {str(e)}")
        continue

plt.tight_layout()
plt.show()